In [11]:
# Notebook til at se på SSI's data for antigentests
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "Dark2"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams['lines.markersize'] = 3
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
# locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


In [2]:
print('Make sure to run "get_data" first, so the most recent data is used')

Make sure to run "get_data" first, so the most recent data is used


In [3]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])

print(latestdir)
print(latestDate)

c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData/ssi_data\SSI_data_2021-04-28
2021-04-28 00:00:00


In [4]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldf = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)

# Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
df = fulldf.iloc[:-2,:].copy()

dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
df["PosPct"] = pd.to_numeric(df["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
df["Date"] = df["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    df[name] = pd.to_numeric(df[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))


In [5]:
curDir = latestdir
# "C:\\Users\\rakrpe\\Documents\\PandemiX\\GithubRepos\\PandemiX\\DanskeData\\ssi_data\\SSI_data_2021-04-28\\"
# latestdir 

In [6]:
df2 = pd.read_csv(curDir+'\\Antigentests_pr_dag.csv',delimiter = ';')

In [7]:
df2["Dato"] = df2["Dato"].astype('datetime64[D]')
df2["Andel_AGpos_PCRkonf_pct"] = pd.to_numeric(df2["Andel_AGpos_PCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))
df2["Andel_AGminusPCRkonf_pct"] = pd.to_numeric(df2["Andel_AGminusPCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))

In [36]:
# "Måske skal vi lave to figurer en med de to øverste paneler (antal test af hver type og fordeling af AG pos på confirm + confirm- og ej confirm Og en anden med False Pos  and false neg rates and specificity Viggo"
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

meanWidth = 7
rmDate = rnTime(df2.Dato,meanWidth)
# Number of tests
ax1.plot(df2.Dato,df2.AG_testede,'k.-',linewidth=0.15,label='AG tests')
ax1.plot(rmDate,rnMean(df2.AG_testede,meanWidth),'k',label=f'{meanWidth} day running mean')
ax1.plot(dfPCR.Date,dfPCR.Tested,'b.-',linewidth=0.15,label='PCR tests')
ax1.plot(rnTime(dfPCR.Date,meanWidth),rnMean(dfPCR.Tested,meanWidth),'b',label=f'{meanWidth} day running mean')


unconf = df2['AGpos_minusPCRkonf']
unconfAndPos = unconf+df2['AGpos_PCRpos']
allAGPos = unconfAndPos+df2['AGposPCRneg']

ax2.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),color='xkcd:olive',label='AG+\nPCR-')
ax2.fill_between(df2.Dato,100 * np.divide(unconfAndPos,allAGPos),color='xkcd:dark green',label='AG+\nPCR+')
ax2.fill_between(df2.Dato,100 * np.divide(unconf,allAGPos),color='gray',label='AG+\nno PCR yet')

# ax2.fill_between(rmDate,100 * np.ones(rmDate.shape),color='xkcd:olive',label='AG+\nPCR-')
# ax2.fill_between(rmDate,100 * rnMean(np.divide(unconfAndPos,allAGPos),meanWidth),color='xkcd:dark green',label='AG+\nPCR+')
# ax2.fill_between(rmDate,100 * rnMean(np.divide(unconf,allAGPos),meanWidth),color='gray',label='AG+\nno PCR yet')

# # ax2.plot(df2.Dato,100 * np.ones(df2.Dato.shape),'.-',linewidth=0.15,label='AG+\nPCR-')
# ax2.plot(df2.Dato,100 * np.divide(unconfAndPos,allAGPos),'k.-',linewidth=0.15,label='AG+\nPCR+')
# ax2.plot(df2.Dato,100 * np.divide(unconf,allAGPos),'k.-',linewidth=0.15,label='AG+\nno PCR yet')

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.set_ylim(bottom=0)
ax2.set_ylim([0,100])

ax1.set_ylabel('Number of tests')
ax2.set_ylabel('Percentage of AG tests [%]')

ax1.grid()
ax2.grid(axis='x')

lastDate = df2.iloc[-1].Dato
# ax1.set_xlim([np.datetime64('2021-02-01'),today])
ax1.set_xlim([np.datetime64('2021-02-01'),lastDate])

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/Agtest_PCRtest_confirmation')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

sumNeg = df2.AGnegPCRneg.values + df2.AGnegPCRpos.values

# meanWidth = 7
rmDate = rnTime(df2.Dato,meanWidth)
rmNegPos = rnMean(np.divide(df2.AGnegPCRpos,sumNeg),meanWidth)


allConf = df2.AGpos_PCRpos + df2.AGposPCRneg
ax1.plot(df2.Dato,100 * np.divide(df2.AGposPCRneg,allConf),'k.-',linewidth=0.15,label='AG+, PCR-')
ax1.plot(rmDate,100 * rnMean(np.divide(df2.AGposPCRneg,allConf),meanWidth),'k',label=f'{meanWidth} day running mean')

ax2.plot(df2.Dato,100 * np.divide(df2.AGnegPCRpos,sumNeg),'k.-',linewidth=0.15,label='AG-, PCR+')
ax2.plot(rmDate,100 * rmNegPos,'k',label=f'{meanWidth} day running mean')


ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.set_ylim(bottom=0,top=100)
ax2.set_ylim(bottom=0)
# ax2.set_ylim([0,100])

# ax1.set_ylabel('Number of tests')
ax1.set_ylabel('Percentage of positive AG tests [%]')
ax2.set_ylabel('Percentage of negative AG tests [%]')

ax1.grid()
ax2.grid()

lastDate = df2.iloc[-1].Dato
# ax1.set_xlim([np.datetime64('2021-02-01'),today])
ax1.set_xlim([np.datetime64('2021-02-01'),lastDate])

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AG_FalsePos_FalseNeg')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:


# # fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
# fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,sharex=True)

# ax1.plot(df2.Dato,df2.AG_testede,label='AG tests')
# ax1.plot(dfPCR.Date,dfPCR.Tested,label='PCR tests')
# # ax1.plot(df2.Dato,df2.AG_pos)
# # ax1.plot(df2.Dato,100*np.divide(df2.AG_pos,df2.AG_testede),label='Andel af positive AG tests')

# # firstToShow = -75
# # ax1.plot(df.Date,posPctAG,label='Positiv procent, antigen')
# # ax1.plot(dfPCR.Date[firstToShow:],posPctPCR[firstToShow:],label='Positiv procent, PCR')


# unconf = df2['AGpos_minusPCRkonf']
# unconfAndPos = unconf+df2['AGpos_PCRpos']
# allAGPos = unconfAndPos+df2['AGposPCRneg']

# # ax2.plot(df2.Dato,df2['AGpos_minusPCRkonf'])
# # ax2.plot(df2.Dato,df2['AGpos_PCRpos'])
# # ax2.plot(df2.Dato,df2['AGposPCRneg'])

# # ax2.plot(df2.Dato,unconf)
# # ax2.plot(df2.Dato,unconfAndPos)
# # ax2.plot(df2.Dato,allAGPos)
# # ax2.plot(df2.Dato,df2.AG_pos)

# # ax2.plot(df2.Dato,np.divide(unconf,allAGPos),label='AG positiv, endnu ikke PCR testet')
# # ax2.plot(df2.Dato,np.divide(unconfAndPos,allAGPos),label='AG positiv, PCR positiv')
# # ax2.plot(df2.Dato,np.divide(allAGPos,allAGPos),label='AG positiv, PCR negativ')

# ax2.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),label='AG positiv, PCR negativ')
# ax2.fill_between(df2.Dato,100 * np.divide(unconfAndPos,allAGPos),label='AG positiv, PCR positiv')
# ax2.fill_between(df2.Dato,100 * np.divide(unconf,allAGPos),color='gray',label='AG positiv, endnu ikke PCR testet')

# allConf = df2.AGpos_PCRpos + df2.AGposPCRneg
# ax3.plot(df2.Dato,100 * np.divide(df2.AGposPCRneg,allConf),label='AG positiv, PCR negativ')
# # ax3.plot(df2.Dato,100 * np.divide(df2.AGpos_PCRpos,allConf),label='AG positiv, PCR positiv')


# # sumNeg = np.sum(df2.AGnegPCRpos.values,df2.AGnegPCRneg.values)
# sumNeg = df2.AGnegPCRneg.values + df2.AGnegPCRpos.values
# # ax3.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),label='AG negativ, PCR positiv')
# # ax3.fill_between(df2.Dato,100 * np.divide(df2.AGnegPCRneg,sumNeg),label='AG negativ, PCR negativ')
# # ax3.fill_between(df2.Dato,df2.AGnegPCRneg)


# # ax3.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),label='AG negativ, PCR negativ')
# # ax3.fill_between(df2.Dato,100 * np.divide(df2.AGnegPCRpos,sumNeg),label='AG negativ, PCR positiv')
# ax4.plot(df2.Dato,100 * np.divide(df2.AGnegPCRpos,sumNeg),label='AG negativ, PCR positiv')

# meanWidth = 7
# rmDate = rnTime(df2.Dato,meanWidth)
# rmNegPos = rnMean(np.divide(df2.AGnegPCRpos,sumNeg),meanWidth)
# ax4.plot(rmDate,100 * rmNegPos,'k:',label='7 dages løbende gennemsnit')


# # ax1.legend(loc='upper left')
# # ax2.legend(loc='upper left')
# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax4.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax1.set_ylim(bottom=0)
# ax2.set_ylim([0,100])
# ax3.set_ylim([0,100])
# ax4.set_ylim([0,100])
# # ax4.set_ylim([95,100])
# ax4.set_ylim([0,3])

# ax1.set_ylabel('Antal tests')
# ax2.set_ylabel('Andel af tests [%]')
# ax3.set_ylabel('Andel af PCR-\nkonfirmede tests [%]')
# ax4.set_ylabel('Andel af tests [%]')
# # ax1.set_xlim([df2.iloc[0,0],df2.iloc[-1,0]])

# ax1.grid()
# # ax2.grid()
# ax3.grid()
# ax4.grid()

# lastDate = df2.iloc[-1].Dato
# # ax1.set_xlim([np.datetime64('2021-02-01'),today])
# ax1.set_xlim([np.datetime64('2021-02-01'),lastDate])

# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
# plt.tight_layout()

# if saveFigures:
#     fig.savefig('figs/AGprDagOpsummering')